## Load Packages

In [1]:
using StatsBase

include("printmat.jl")

println4Ps (generic function with 1 method)

In [2]:
using Plots
backend = "gr"              #"gr" (default), "pyplot" 

if backend == "pyplot"
    pyplot(size=(600,400))
else    
    gr(size=(600,400))
end

Plots.GRBackend()

# Loading Daily S&P 500 Data

In [3]:
xx = readdlm("Data/SP500RfPs.csv",',',header=true)
x  = xx[1]
SP = convert(Array{Float64},x[:,2])      #convert to numerical array, S&P 500 level
R  = (SP[2:end]./SP[1:end-1] - 1) * 100  #returns, % 
T  = length(R)  
println("Number of days in the sample: ",T)

dN = Array{Date}(T)                      #convert to Date
for t = 1:T
    dN[t] = Date(x[t+1,1],"d/m/y")
end

Number of days in the sample: 9352


# Autocorrelations

$\rho_s = \text{Corr}(R_t,R_{t-s})$

In large samples, $\sqrt{T}\hat{\rho}_{s}\sim N(0,1)$

In [4]:
plags = 1:5
ρ = autocor(R,plags)
println("\nlag, autocorrr, t-stat(autocorr) of excess returns")
printmat([plags ρ sqrt(T)*ρ])


lag, autocorrr, t-stat(autocorr) of excess returns
     1.000    -0.026    -2.480
     2.000    -0.044    -4.208
     3.000    -0.007    -0.684
     4.000    -0.022    -2.124
     5.000    -0.018    -1.744



# Autoregressions

An AR(1) is 

$R_{t}=c+a_{1}R_{t-1}+\varepsilon_{t}$

and can be estimated by OLS

An asymmteric AR(1) 

$R_{t} =\alpha+\beta Q_{t-1}R_{t-1}+\gamma(1-Q_{t-1})R_{t-1}+\varepsilon
_{t}$,

where $Q_{t-1}=1 \ \text{ if } \ R_{t-1}<0$ and zero otherwise

In [5]:
function OlsFn(y,x)                      #OLS estimation 
    T = size(y,1)                        #alternative: use GLM.jl
    b     = x\y
    yhat  = x*b
    res   = y - yhat
    Covres = cov(res)
    Covb   = kron(Covres,inv(x'x))       #works also if y is Txn
    R2a    = 1 - var(res,1)./var(y,1)
    return b,res,yhat,Covb,R2            #the outputs
end

OlsFn (generic function with 1 method)

In [6]:
y  = R[2:end]
Tb = size(y,1)
x  = [ones(Tb) R[1:end-1]]
(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt.(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1): intercept and slope with t-stats")
printmat([b tstat])

Results from AR(1): intercept and slope with t-stats
     0.039     3.405
    -0.026    -2.481



In [7]:
DummyNegative = R[1:end-1] .< 0

x = [ones(Tb) DummyNegative.*R[1:end-1] (1-DummyNegative).*R[1:end-1]]

(b,res,yhat,Covb,) = OlsFn(y,x) 
Stdb  = sqrt.(diag(Covb))
tstat = b./Stdb 

println("Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats")
printmat([b tstat])

Results from AR(1) with dummies: [intercept, slope neg, slope pos] with t-stats
     0.001     0.058
    -0.075    -4.395
     0.027     1.518



## Recursive Estimation and Out-of-Sample R2

Do recursive estimation (longer and longer sample) an predict one period ahead (outside of the sample). Define an "out-of-sample" $R^2$ as 

$R_{OS}^2 = 1- \frac{\text{MSE(forecasting model)}}{\text{MSE(benchmark forecast)}}$

In [8]:
y = R[2:end]
Tb = size(y,1)
x = [ones(Tb,1) R[1:end-1]]

bT     = fill(NaN,size(x))
rHat   = fill(NaN,Tb)
rTilde = fill(NaN,Tb)
for t = 100:Tb
    (b,res,yhat,Covb,) = OlsFn(y[1:t-1],x[1:t-1,:]) 
    bT[t,:] = b'
    rHat[t] = (x[t:t,:]*b)[1]           #out-of-sample forecast of period t
    rTilde[t] = mean(y[1:t-1])          #benchmark forecast: historical average
end    

MSE_Model = mean((y[100:end]-rHat[100:end]).^2)    #MSE for out-of-sample period
MSE_Bench = mean((y[100:end]-rTilde[100:end]).^2)

printlnPs("MSE of AR(1) model and the benchmark: ",[MSE_Model MSE_Bench])
printlnPs("out-of-sample R2: ",1-MSE_Model/MSE_Bench)

MSE of AR(1) model and the benchmark:      1.247     1.244
out-of-sample R2:     -0.002


In [9]:
#YearFrac  = Dates.year.(dN) + (Dates.month.(dN)-1)/12 + (Dates.day.(dN)-1)/(31*12)

plot(dN[2:end],bT[:,1],color=:blue,legend=false)
title!("Estimated intercept")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980-01-01 
 
 
 1989-01-01 
 
 
 1998-01-01 
 
 
 2007-01-01 
 
 
 2016-01-01 
 
 
 0.02 
 
 
 0.04 
 
 
 0.06 
 
 
 Estimated intercept 
 
<polyline clip-path="url(#clip02)" style="stroke:#0000ff; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.1267,271.577 45.29,257.165 45.3308,268.323 45.3716,319.737 45.4125,318.47 45.5349,314.281 45.5758,307.528 45.6166,242.473 45.6574,217.897 45.6983,199.617 
 45.8207,217.295 45.8616,196.343 45.9024,156.702 45.9432,184.509 45.984,189.749 46.1065,196.303 46.1474,222.504 46.1882,220.817 46.229,220.041 46.2698,200.884 
 46.3923,180.756 46.4331,207.024 46.474,226.683 46.5148,198.289 46.5556,180.496 46.6781,179.542 46.7189,220.058 46.8006,213.161 46.8414,200.757 46.9639,155.888 
 47.0047,132.203 47.0455,146.577 47.0864,170.708 47.1272,210.823 47.2497,223.953 47.2905,206.039 47.3313,243.123 47.3722,246.06 47.413,249.667 47.5355,242.125 
 47.5763,252.684 47.6171,237.837 47.6579,200.004 47.6988,204.646 47.8213,206.013 47.8621,205.448 47.9029,183.004 47.9437,174.894 47.9846,180.053 48.107,183.343 
 48.1479,175.268 48.1887,132.217 48.2295,128.984 48.2704,147.68 48.3928,114.193 48.4337,89.5343 48.4745,91.8054 48.5153,70.602 48.5561,79.3344 48.6786,73.8002 
 48.7194,60.208 48.7603,61.5811 48.8011,61.4251 48.8419,74.7816 48.9644,76.4636 49.0052,61.5272 49.0461,68.3483 49.0869,69.919 49.1277,71.8521 49.291,64.7502 
 49.3319,120.309 49.3727,155.072 49.4135,136.364 49.536,116.835 49.5768,107.846 49.6176,128.733 49.6585,117.754 49.6993,119.461 49.8218,95.1013 49.8626,97.2975 
 49.9034,122.528 49.9443,112.03 49.9851,54.871 50.1076,61.2651 50.1484,86.5176 50.1892,83.1822 50.23,77.5589 50.2709,73.5946 50.3934,98.6594 50.4342,120.568 
 50.475,88.8134 50.5158,92.6527 50.5567,79.0222 50.6791,56.1819 50.72,88.9005 50.7608,188.365 50.8016,220.644 50.8424,222.145 50.9649,236.899 51.0058,266.151 
 51.0466,265.732 51.0874,260.178 51.1282,256.098 51.2507,309.056 51.2915,325.089 51.3324,332.558 51.3732,326.28 51.414,338.839 51.5365,321.616 51.5773,321.152 
 51.6182,271.983 51.659,299.561 51.6998,276.383 51.8223,281.209 51.8631,298.694 51.9039,312.316 51.9448,344.263 51.9856,327.193 52.1081,299.58 52.1489,258.74 
 52.1897,279.011 52.2306,265.953 52.2714,251.175 52.3939,262.159 52.4347,250.809 52.4755,265.226 52.5572,258.845 52.6797,242.356 52.7205,199.859 52.7613,215.07 
 52.8021,205.271 52.843,206.53 52.9654,221.742 53.0063,227.763 53.0471,205.542 53.0879,200.15 53.1288,186.927 53.2512,199.695 53.2921,195.486 53.3329,200.558 
 53.3737,200.099 53.4145,197.827 53.537,173.237 53.5779,170.243 53.6187,192.663 53.6595,192.164 53.7003,191.404 53.8228,206.016 53.9045,203.005 53.9453,201.54 
 53.9861,199.051 54.1086,202.739 54.1903,201.073 54.2311,245.349 54.2719,250.873 54.3944,222.201 54.4352,221.214 54.476,181.152 54.5169,185.626 54.5577,170.593 
 54.6802,173.138 54.721,165.309 54.7618,153.679 54.8027,158.113 54.8435,165.332 54.966,158.032 55.0068,141.522 55.0476,154.588 55.0884,116.335 55.1293,117.411 
 55.2518,120.595 55.2926,99.0779 55.3334,114.354 55.3742,91.868 55.4151,110.704 55.5375,90.7593 55.5784,104.938 55.6192,99.0998 55.66,82.9973 55.7009,77.3648 
 55.8233,53.1099 55.8642,67.5121 55.905,52.9334 55.9458,47.1734 55.9866,75.3816 56.15,99.2403 56.1908,113.055 56.2316,78.633 56.2724,97.4466 56.3949,100.992 
 56.4357,130.423 56.4766,115.763 56.5174,142.169 56.5582,141.146 56.6807,120.211 56.7215,139.14 56.7624,133.421 56.8032,160.839 56.844,205.932 56.9665,236.576 
 57.0073,240.807 57.0481,218.454 57.089,235.167 57.1298,256.189 57.2523,257.61 57.2931,311.905 57.3339,269.899 57.3748,269.29 57.4156,289.928 57.5381,303.073 
 57.5789,355.598 57.6197,352.152 57.6605,361.095 57.7014,368.504 57.8239,324.176 57.8647,305.271 57.9055,306.435 57.9463,298.473 58.1096,308.273 58.1505,340.88 
 58.1913,319.582 58.2321,290.251 58.273,278.77 58.3954,285.287 58.4363,300.632 58.4771,302.71 58.5179,320.437 58.5587,

In [10]:
plot(dN[2:end],bT[:,2],color=:red,legend=false)
title!("Estimated AR(1) slope")

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1980-01-01 
 
 
 1989-01-01 
 
 
 1998-01-01 
 
 
 2007-01-01 
 
 
 2016-01-01 
 
 
 0.00 
 
 
 0.05 
 
 
 0.10 
 
 
 0.15 
 
 
 Estimated AR(1) slope 
 
<polyline clip-path="url(#clip02)" style="stroke:#ff0000; stroke-width:1; stroke-opacity:1; fill:none" points="
 45.1267,215.556 45.29,215.498 45.3308,217.639 45.3716,209.92 45.4125,210.769 45.5349,210.941 45.5758,210.609 45.6166,204.423 45.6574,178.23 45.6983,169.097 
 45.8207,175.103 45.8616,180.213 45.9024,168.867 45.9432,189.265 45.984,187.173 46.1065,186.413 46.1474,183.371 46.1882,184.048 46.229,184.062 46.2698,183.983 
 46.3923,177.332 46.4331,187.974 46.474,179.666 46.5148,189.43 46.5556,181.177 46.6781,180.804 46.7189,182.597 46.8006,187.158 46.8414,186.63 46.9639,176.065 
 47.0047,151.743 47.0455,161.378 47.0864,155.688 47.1272,138.61 47.2497,129.874 47.2905,135.238 47.3313,146.241 47.3722,144.327 47.413,143.842 47.5355,144.52 
 47.5763,145.941 47.6171,148.954 47.6579,138.5 47.6988,142.861 47.8213,142.832 47.8621,142.852 47.9029,142.81 47.9437,138.639 47.9846,139.918 48.107,139.607 
 48.1479,140.311 48.1887,133.103 48.2295,129.08 48.2704,133.035 48.3928,146.081 48.4337,127.637 48.4745,129.578 48.5153,129.176 48.5561,134.021 48.6786,134.966 
 48.7194,133.588 48.7603,134.096 48.8011,134.096 48.8419,134.074 48.9644,133.554 49.0052,134.776 49.0461,137.428 49.0869,137.194 49.1277,137.075 49.291,137.51 
 49.3319,147.514 49.3727,108.93 49.4135,122.284 49.536,116.166 49.5768,110.416 49.6176,117.884 49.6585,123.255 49.6993,123.581 49.8218,124.285 49.8626,125.912 
 49.9034,126.033 49.9443,132.284 49.9851,122.799 50.1076,139.036 50.1484,136.896 50.1892,138.802 50.23,138.828 50.2709,138.232 50.3934,141.412 50.4342,128.378 
 50.475,145.133 50.5158,148.261 50.5567,148.971 50.6791,139.961 50.72,168.776 50.7608,87.2392 50.8016,38.3352 50.8424,37.1779 50.9649,34.4344 51.0058,23.5969 
 51.0466,23.8862 51.0874,24.6227 51.1282,24.1772 51.2507,30.6629 51.2915,12.0825 51.3324,7.87402 51.3732,9.68035 51.414,10.8397 51.5365,15.9434 51.5773,15.77 
 51.6182,11.8969 51.659,50.8507 51.6998,62.9577 51.8223,65.2497 51.8631,64.2637 51.9039,58.3648 51.9448,45.7561 51.9856,59.5314 52.1081,52.1605 52.1489,19.656 
 52.1897,47.3968 52.2306,51.8843 52.2714,48.1049 52.3939,52.9418 52.4347,55.4032 52.4755,58.9699 52.5572,61.0697 52.6797,59.4202 52.7205,39.4369 52.7613,61.8382 
 52.8021,64.3298 52.843,64.5825 52.9654,64.5095 53.0063,62.2492 53.0471,66.8912 53.0879,63.8101 53.1288,60.5818 53.2512,66.0221 53.2921,67.2329 53.3329,67.5231 
 53.3737,67.5826 53.4145,67.6087 53.537,66.1751 53.5779,63.8396 53.6187,68.5151 53.6595,68.7926 53.7003,68.8581 53.8228,68.9246 53.9045,70.0979 53.9453,70.0799 
 53.9861,69.9765 54.1086,70.2609 54.1903,70.4239 54.2311,71.7504 54.2719,65.7207 54.3944,75.0086 54.4352,74.2061 54.476,68.402 54.5169,74.884 54.5577,74.4954 
 54.6802,75.683 54.721,75.7976 54.7618,73.1224 54.8027,74.9515 54.8435,74.3647 54.966,75.9395 55.0068,73.0281 55.0476,80.7523 55.0884,93.4849 55.1293,94.8455 
 55.2518,95.1716 55.2926,96.9736 55.3334,107.889 55.3742,117.047 55.4151,128.961 55.5375,139.208 55.5784,146.52 55.6192,148.802 55.66,146.927 55.7009,143.661 
 55.8233,137.255 55.8642,151.817 55.905,157.848 55.9458,155.628 55.9866,162.462 56.15,144.81 56.1908,136.496 56.2316,150.524 56.2724,171.828 56.3949,169.882 
 56.4357,165.448 56.4766,176.432 56.5174,184.636 56.5582,185.366 56.6807,186.309 56.7215,199.723 56.7624,202.823 56.8032,205.876 56.844,172.669 56.9665,140.636 
 57.0073,137.287 57.0481,141.937 57.089,153.068 57.1298,143.78 57.2523,142.936 57.2931,136.943 57.3339,194.707 57.3748,194.124 57.4156,195.929 57.5381,188.671 
 57.5789,168.243 57.6197,172.854 57.6605,172.421 57.7014,170.585 57.8239,180.571 57.8647,157.258 57.9055,158.033 57.9463,157.783 58.1096,159.944 58.1505,151.947 
 58.1913,170.029 58.2321,156.381 58.273,146.047 58.3954,148.835 58.4363,146.753 58.4771,145.901 58.51